In [7]:
import yfinance as yf
import pymysql
import datetime 
import pandas as pd 
from dateutil.relativedelta import relativedelta
from yahoo_earnings_calendar import YahooEarningsCalendar
yec = YahooEarningsCalendar()

In [9]:
# 1. DB 연결
def connect_db():
    conn = pymysql.connect(host="localhost", user="root", password="4team123!",
                           db="investar", charset="utf8")  
    return conn

today = datetime.datetime.now().strftime('%Y%m%d') 
conn = connect_db()
curs = conn.cursor() 

# 종목(Ticker) 리스트 가져오기

In [4]:
sql = "SELECT DISTINCT symbol FROM USA_STOCK_INFO"
curs.execute(sql) 
stock_list = [item[0] for item in curs.fetchall()] 
# conn.commit() 
# conn.close() 
print(len(stock_list))

7525


In [5]:
# 파일 다운로드 함수 
file_path = "./ASSETS/" 
def download(url, file_name = None): 
    if not file_name: 
        file_name = url.split('/')[-1] 
        
    with open(file_path+file_name, "wb") as file: 
        response = requests.get(url) 
        if response.status_code == 200: 
            file.write(response.content) 
            return True 
        return False

In [6]:
# 실적 정보 다운로드 함수 
def download_data(ticker): 
    income_url = "https://stockrow.com/api/companies/{ticker}/financials.xlsx?dimension=Q&section=Income%20Statement&sort=desc" 
    # Ticker 실적 정보 다운로드 
    income_down = download(income_url.format(ticker=ticker), "{ticker}_INCOME.xlsx".format(ticker=ticker)) 
    if income_down == True: 
        return True 
    else: 
        return False

In [7]:
# 실적 정보 가져오는 함수 
def get_stock_data(ticker): 
    # Ticker 주식 정보 Read 
    df_income = pd.read_excel(file_path+"{ticker}_INCOME.xlsx".format(ticker=ticker), index_col=0).fillna(0) 
    df_income = df_income.T 
    
    list_isnert = [] 
    null_value = float(0) 
    
    # 매출액 최근 5분기 
    if 'Revenue' in df_income.columns: 
        for i in range(0,5): 
            list_isnert.append(float(df_income.iloc[i]['Revenue'])) 
    else : 
        list_isnert = list_isnert + [0,0,0,0,0] 
        
    # 영업이익 최근 5분기 
    if 'Operating Income' in df_income.columns: 
        for i in range(0,5): 
            list_isnert.append(float(df_income.iloc[i]['Operating Income'])) 
    else: 
        list_isnert = list_isnert + [0,0,0,0,0] 
        
    # 순이익 최근 5분기 
    if 'Net Income Common' in df_income.columns: 
        for i in range(0,5): 
            list_isnert.append(float(df_income.iloc[i]['Net Income Common'])) 
    else: 
        list_isnert = list_isnert + [0,0,0,0,0] 
        list_isnert = list_isnert + [str(df_income.index[0]).replace('-','')[:8], ticker] 
    return list_isnert

In [9]:
sql = """UPDATE USA_STOCK_INFO SET SR_RE1 = %s , SR_RE2 = %s , SR_RE3 = %s , SR_RE4 = %s , SR_RE5 = %s , SR_OI1 = %s , SR_OI2 = %s , SR_OI3 = %s , SR_OI4 = %s , SR_OI5 = %s , SR_NI1 = %s , SR_NI2 = %s , SR_NI3 = %s , SR_NI4 = %s , SR_NI5 = %s , SR_YMD = %s WHERE STOCK_CODE = %s """

In [10]:
print(sql)

UPDATE USA_STOCK_INFO SET SR_RE1 = %s , SR_RE2 = %s , SR_RE3 = %s , SR_RE4 = %s , SR_RE5 = %s , SR_OI1 = %s , SR_OI2 = %s , SR_OI3 = %s , SR_OI4 = %s , SR_OI5 = %s , SR_NI1 = %s , SR_NI2 = %s , SR_NI3 = %s , SR_NI4 = %s , SR_NI5 = %s , SR_YMD = %s WHERE STOCK_CODE = %s 


In [ ]:
# 반복문 돌면서 INSERT 

for i, ticker in enumerate(stock_list): 
    try: 
        if download_data(ticker): 
            stock_info = get_stock_data(ticker) 
            curs.execute(sql, stock_info) 
            conn.commit() 
        # else: 
        # print(i, ' 번 째 ', ticker, ' : NO DATA PASS') 
    except Exception as e: 
        print(i, ' 번 째 오류 발생 : ', ticker, ' 오류:', str(e)) conn.close()